ok i have sent you an algorithm in image attachment. check that and add those features in the code im providing also it will be helpful if you can add some thing from your side which can be efficient for detection :

here is the code where you have to add also make it as a function whatever you have adding so that will be easy for reading and editing :



In [ ]:
import cv2
import numpy as np
import math

# Load and preprocess the image
img = cv2.imread(r'C:\Users\ashut\Documents\GitHub\ProjectMajor\diagram_segment.png')

if img is None:
    print("Image not found or path is incorrect!")
    exit()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 1)
edges = cv2.Canny(blurred, 50, 150)

cv2.imshow("Edges", edges)  # See if edges are being detected properly

# Find contours and hierarchy
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

if len(contours) == 0:
    print("No contours found!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

if hierarchy is None:
    print("No hierarchy returned!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

# simplify the hierarchy array
hierarchy = hierarchy[0]

shapes = []

# Loop through contours
for i, cnt in enumerate(contours):

    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    vertices = len(approx)

    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = 0, 0
    center = (cx, cy)

    area = cv2.contourArea(cnt)

    side_lengths = []
    for j in range(vertices):
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]
        dist = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
        side_lengths.append(dist)

    angles = []
    for j in range(vertices):
        p0 = approx[j - 1][0]
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]

        v1 = (p0[0] - p1[0], p0[1] - p1[1])
        v2 = (p2[0] - p1[0], p2[1] - p1[1])

        dot_prod = v1[0]*v2[0] + v1[1]*v2[1]
        mag1 = math.hypot(v1[0], v1[1])
        mag2 = math.hypot(v2[0], v2[1])

        if mag1 == 0 or mag2 == 0:
            angle = 0
        else:
            cos_theta = dot_prod / (mag1 * mag2)
            cos_theta = max(min(cos_theta, 1), -1)
            angle = math.acos(cos_theta) * 180 / math.pi

        angles.append(angle)

    shape = "Unknown"
    
    if vertices == 3:
        shape = "Triangle"
    elif vertices == 4:
        sides_equal = all(abs(side_lengths[k] - side_lengths[0]) < 5 for k in range(1, 4))
        angles_close_90 = all(abs(a - 90) < 10 for a in angles)

        if sides_equal and angles_close_90:
            shape = "Square"
        elif angles_close_90:
            shape = "Rectangle"
        else:
            shape = "Quadrilateral"
    elif vertices > 4:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        circularity = 4 * math.pi * area / (cv2.arcLength(cnt, True) ** 2)

        if circularity > 0.8:
            shape = "Circle"
        else:
            shape = f"{vertices}-gon"

    #print(f"Detected: {shape}")
    print(shape)
    shapes.append({
        'index': i,
        'shape': shape,
        'center': center,
        'area': area,
        'parent': hierarchy[i][3],
        'contour': approx,
        'sides': side_lengths,
        'angles': angles
    })


'''  inscribed shape formatter '''

for s in shapes:
    if s['parent'] != -1:
        parent = shapes[s['parent']]
        dx = abs(s['center'][0] - parent['center'][0])
        dy = abs(s['center'][1] - parent['center'][1])
        area_ratio = s['area'] / parent['area']
        print(f"Checking: {s['shape']} inside {parent['shape']} -> dx: {dx}, dy: {dy}, area_ratio: {area_ratio}")
        if dx < 10 and dy < 10:
            if 0.3 < area_ratio < 0.7:
                print(f"{s['shape']} is inscribed in {parent['shape']}")

for s in shapes:
    cv2.drawContours(img, [s['contour']], -1, (0, 255, 0), 2)
    cv2.putText(img, s['shape'], s['center'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
cv2.imshow('shapes', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


6-gon
6-gon
7-gon
7-gon
6-gon
6-gon
8-gon
Square
Square
8-gon
8-gon
10-gon
10-gon
Circle
Circle
8-gon
6-gon
6-gon
8-gon
8-gon
Circle
Circle
9-gon
6-gon
6-gon
Checking: Square inside Square -> dx: 0, dy: 0, area_ratio: 0.9962462462462462
Checking: 8-gon inside 8-gon -> dx: 0, dy: 0, area_ratio: 0.9965286580728202
Checking: 10-gon inside 8-gon -> dx: 18, dy: 30, area_ratio: 0.013835887199358108
Checking: 10-gon inside 10-gon -> dx: 0, dy: 0, area_ratio: 0.983026874115983
Checking: Circle inside Circle -> dx: 0, dy: 0, area_ratio: 0.8987341772151899
Checking: 6-gon inside 6-gon -> dx: 0, dy: 0, area_ratio: 0.9966801495840647
Checking: 8-gon inside 6-gon -> dx: 79, dy: 31, area_ratio: 0.017152264634940084
Checking: 8-gon inside 8-gon -> dx: 0, dy: 0, area_ratio: 0.9776785714285714
Checking: Circle inside 8-gon -> dx: 2, dy: 3, area_ratio: 0.3070776255707763
Circle is inscribed in 8-gon
Checking: Circle inside Circle -> dx: 0, dy: 0, area_ratio: 0.9553903345724907


# Claude 

In [14]:
import cv2
import numpy as np
import math

def detect_shapes(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  #loading image
    if img is None:
        print("Image not found or path is incorrect!")
        return None


    # Apply multiple preprocessing techniques
    # Try different thresholding methods
    methods = [
        cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU),
        cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV),
        cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY_INV, 11, 2)
    ]

    for _, binary in methods:
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter and analyze contours
        circle_contours = []
        for cnt in contours:
            # Skip very small contours
            if cv2.contourArea(cnt) < 100:
                continue

            # Fit an ellipse to the contour
            if len(cnt) >= 5:
                try:
                    (x,y), (MA,ma), angle = cv2.fitEllipse(cnt)
                except:
                    continue

            # Compute circularity
            area = cv2.contourArea(cnt)
            perimeter = cv2.arcLength(cnt, True)
            
            if perimeter == 0:
                continue
            
            circularity = 4 * math.pi * area / (perimeter ** 2)

            # Check if it's very circular
            if circularity > 0.8:
                circle_contours.append((cnt, circularity))

        # Sort circle contours by area (largest first)
        if circle_contours:
            circle_contours.sort(key=lambda x: cv2.contourArea(x[0]), reverse=True)
            
            main_circle = circle_contours[0][0]
            
            # Visualize and find inner shapes
            result_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            shapes_inside = []

            # Add main circle
            M = cv2.moments(main_circle)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                shapes_inside.append({
                    'shape': 'Circle',
                    'contour': main_circle,
                    'center': (cx, cy)
                })
                
                # Draw main circle
                cv2.drawContours(result_img, [main_circle], -1, (0, 255, 0), 2)

            # Find shapes inside main circle
            for cnt in contours:
                # Skip main circle and small contours
                if np.array_equal(cnt, main_circle) or cv2.contourArea(cnt) < 50:
                    continue

                # Check if inside main circle
                M = cv2.moments(cnt)
                if M['m00'] == 0:
                    continue

                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])

                # Point-in-polygon test
                if cv2.pointPolygonTest(main_circle, (cx, cy), False) >= 0:
                    # Approximate contour
                    epsilon = 0.04 * cv2.arcLength(cnt, True)
                    approx = cv2.approxPolyDP(cnt, epsilon, True)

                    # Determine shape
                    shape = determine_shape(approx, cnt)
                    
                    shapes_inside.append({
                        'shape': shape,
                        'contour': approx,
                        'center': (cx, cy)
                    })

                    # Draw inner shapes
                    cv2.drawContours(result_img, [approx], -1, (0, 255, 0), 2)
                    cv2.putText(result_img, shape, (cx, cy), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

            # Display results
            cv2.imshow('Detected Shapes', result_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            return shapes_inside

    print("No main circle detected!")
    return None

def determine_shape(approx, cnt):
    vertices = len(approx)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    # Circularity check
    circularity = 4 * math.pi * area / (perimeter ** 2)

    # Shape determination logic
    if vertices == 3:
        return "Triangle"
    elif vertices == 4:
        # Check for square or rectangle
        side_lengths = [math.hypot(approx[j][0][0] - approx[(j+1)%4][0][0], 
                                    approx[j][0][1] - approx[(j+1)%4][0][1]) 
                        for j in range(4)]
        
        # Calculate angle deviation from 90 degrees
        angles = []
        for j in range(4):
            p0 = approx[j-1][0]
            p1 = approx[j][0]
            p2 = approx[(j+1)%4][0]
            
            v1 = p0 - p1
            v2 = p2 - p1
            
            angle = math.degrees(math.atan2(np.linalg.det([v1,v2]), np.dot(v1,v2)))
            angles.append(abs(angle))
        
        # Check for square
        if (max(side_lengths) - min(side_lengths) < 10 and 
            all(abs(a - 90) < 10 for a in angles)):
            return "Square"
        # Check for rectangle
        elif all(abs(a - 90) < 15 for a in angles):
            return "Rectangle"
        else:
            return "Quadrilateral"
    
    # Circle check
    elif circularity > 0.8:
        return "Circle"
    elif vertices > 4:
        return f"{vertices}-gon"
    
    return "Unknown Shape"

# Main execution
if __name__ == "__main__":
    image_path = r'C:\Users\ashut\Documents\GitHub\ProjectMajor\grayscale_image.png'
    
    # Detect shapes
    shapes_inside = detect_shapes(image_path)
    
    if shapes_inside:
        print("\nShapes detected:")
        for shape in shapes_inside:
            print(f"Shape: {shape['shape']}")
    else:
        print("No shapes found.")


Shapes detected:
Shape: Circle


# How come upper one is giving me perfect circle and down one is giving me wrong circle??

In [16]:
import cv2
import numpy as np
import math

def detect_shapes(image_path, min_shape_area=50, max_shapes=10):
    # Load and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print("Image not found or path is incorrect!")
        return None

    # Preprocessing methods
    def process_image(image):
        # Try different thresholding methods
        methods = [
            cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU),
            cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV),
            cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)
        ]

        for method in methods:
            if isinstance(method, tuple):
                _, binary = method
            else:
                binary = method

            # Find contours
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Filter contours
            valid_contours = []
            for cnt in contours:
                # Skip very small contours
                if cv2.contourArea(cnt) < min_shape_area:
                    continue
                valid_contours.append(cnt)

            # Sort contours by area in descending order
            valid_contours.sort(key=lambda x: cv2.contourArea(x), reverse=True)

            # Limit number of shapes
            valid_contours = valid_contours[:max_shapes]

            # Visualize and analyze shapes
            result_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            shapes_detected = []

            for cnt in valid_contours:
                # Approximate contour
                epsilon = 0.04 * cv2.arcLength(cnt, True)
                approx = cv2.approxPolyDP(cnt, epsilon, True)

                # Calculate moments for center
                M = cv2.moments(cnt)
                if M['m00'] == 0:
                    continue

                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])

                # Determine shape
                shape = determine_shape(approx, cnt)
                
                shapes_detected.append({
                    'shape': shape,
                    'contour': approx,
                    'center': (cx, cy),
                    'area': cv2.contourArea(cnt)
                })

                # Draw shapes
                cv2.drawContours(result_img, [approx], -1, (0, 255, 0), 2)
                cv2.putText(result_img, f"{shape} ({cv2.contourArea(cnt):.0f})", 
                            (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

            # Display results
            cv2.imshow('Detected Shapes', result_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            return shapes_detected

        return None

    # Try processing the image
    shapes_detected = process_image(img)
    
    if shapes_detected is None:
        print("No shapes detected!")
        return None
    
    return shapes_detected

def determine_shape(approx, cnt):
    vertices = len(approx)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    # Circularity check
    circularity = 4 * math.pi * area / (perimeter ** 2)

    # Shape determination logic
    if vertices == 3:
        return "Triangle"
    elif vertices == 4:
        # Check for square or rectangle
        side_lengths = [math.hypot(approx[j][0][0] - approx[(j+1)%4][0][0], 
                                    approx[j][0][1] - approx[(j+1)%4][0][1]) 
                        for j in range(4)]
        
        # Calculate angle deviation from 90 degrees
        angles = []
        for j in range(4):
            p0 = approx[j-1][0]
            p1 = approx[j][0]
            p2 = approx[(j+1)%4][0]
            
            v1 = p0 - p1
            v2 = p2 - p1
            
            angle = math.degrees(math.atan2(np.linalg.det([v1,v2]), np.dot(v1,v2)))
            angles.append(abs(angle))
        
        # Check for square
        if (max(side_lengths) - min(side_lengths) < 10 and 
            all(abs(a - 90) < 10 for a in angles)):
            return "Square"
        # Check for rectangle
        elif all(abs(a - 90) < 15 for a in angles):
            return "Rectangle"
        else:
            return "Quadrilateral"
    
    # Circle check
    elif circularity > 0.8:
        return "Circle"
    elif vertices > 4:
        return f"{vertices}-gon"
    
    return "Unknown Shape"

# Main execution
if __name__ == "__main__":
    image_path = r'C:\Users\ashut\Documents\GitHub\ProjectMajor\Geoimgs\2.png'  # Replace with your image path
    
    # Detect shapes
    shapes_detected = detect_shapes(image_path, min_shape_area=50, max_shapes=10)
    
    if shapes_detected:
        print("\nShapes detected:")
        for shape in shapes_detected:
            print(f"Shape: {shape['shape']}, Area: {shape['area']:.0f}")
    else:
        print("No shapes found.")


Shapes detected:
Shape: Triangle, Area: 60424
Shape: Quadrilateral, Area: 227
Shape: Quadrilateral, Area: 196
Shape: 5-gon, Area: 116
Shape: 10-gon, Area: 98
Shape: 9-gon, Area: 79
Shape: 9-gon, Area: 71
Shape: 7-gon, Area: 61
